In [2]:
import spacy

In [3]:
#!python -m spacy download en_core_web_md

In [4]:
nlp = spacy.load('en_core_web_md')

In [5]:
import wikipedia

text = wikipedia.summary("United States")

In [6]:
doc = nlp(text)

In [14]:
sentence1 = list(doc.sents)[0]
print (sentence1)

The United States of America (USA or U.S.A.), commonly known as the United States (US or U.S.) or America, is a country primarily located in North America.


In [15]:
import numpy as np

In [16]:
word_to_search = 'Country'

In [17]:
ms = nlp.vocab.vectors.most_similar(
    np.asarray([nlp.vocab.vectors[nlp.vocab.strings[word_to_search]]]), n=10)
words = [nlp.vocab.strings[w] for w in ms[0][0]]
distance = ms[2]
print(words)

['Intercountry', 'Strathtay', 'Southerns', 'Heathlands', 'Facies', 'Communitas', 'Betties', 'Norteños', 'Trencsén', 'Regionalverkehr']


In [18]:
doc1= nlp('I enjoy apples')
doc2= nlp('I enjoy climbing trees')
print(doc1, '<-->', doc2, doc1.similarity(doc2))

I enjoy apples <--> I enjoy climbing trees 0.9021378059432545


# Entity Ruler

In [19]:
nlp = spacy.load('en_core_web_sm')

In [22]:
from spacy.vocab import Vocab
vocab = Vocab(strings=["hello", "world"])
vocab

In [23]:
from spacy.tokens import Doc
from spacy.training import Example

predicted = Doc(vocab, words=["Apply", "some", "sunscreen"])
token_ref = ["Apply", "some", "sun", "screen"]
tags_ref = ["VERB", "DET", "NOUN", "NOUN"]
example = Example.from_dict(predicted, {"words": token_ref, "tags": tags_ref})

In [24]:
example

{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'O'], 'spans': {}, 'links': {}}, 'token_annotation': {'ORTH': ['Apply', 'some', 'sun', 'screen'], 'SPACY': [True, True, True, True], 'TAG': ['VERB', 'DET', 'NOUN', 'NOUN'], 'LEMMA': ['', '', '', ''], 'POS': ['', '', '', ''], 'MORPH': ['', '', '', ''], 'HEAD': [0, 1, 2, 3], 'DEP': ['', '', '', ''], 'SENT_START': [1, 0, 0, 0]}}

# Trade Categorization

In [25]:
import spacy
from spacy.tokens import Span
from spacy.training import Example

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Example training data for NER
train_data = [
    ("The APR is 5% and the up-front points are 2", {'entities': [(4, 7, 'APR'), (24, 27, 'PERCENTAGE')]}),
    ("The fees are $3000", {'entities': [(4, 8, 'FEES')]}),
    ("Up-front points of 1.5", {'entities': [(0, 16, 'UP_FRONT_POINTS'), (20, 23, 'POINTS')]}),
    # Add more training data
]

# Adding labels to the NER model
ner = nlp.get_pipe("ner")
ner.add_label("APR")
ner.add_label("FEES")
ner.add_label("UP_FRONT_POINTS")
ner.add_label("POINTS")
ner.add_label("PERCENTAGE")

# Disable other pipelines while training NER
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for itn in range(10):
        random.shuffle(train_data)
        losses = {}
        for text, annotations in train_data:
            example = Example.from_dict(nlp.make_doc(text), annotations)
            nlp.update([example], drop=0.5, sgd=optimizer, losses=losses)
        print(losses)

# Function to extract labels and values
def extract_trade_info(text):
    doc = nlp(text)
    trade_info = {}
    for ent in doc.ents:
        trade_info[ent.label_] = ent.text
    return trade_info

# Example rules for categorizing trades based on extracted values
def categorize_trade(trade_info):
    if 'APR' in trade_info:
        apr_value = float(trade_info['APR'].strip('%'))
        if apr_value < 3:
            return 'Low APR'
        elif 3 <= apr_value <= 5:
            return 'Medium APR'
        else:
            return 'High APR'
    if 'FEES' in trade_info:
        fees_value = float(trade_info['FEES'].strip('$'))
        if fees_value < 1000:
            return 'Low Fees'
        elif 1000 <= fees_value <= 3000:
            return 'Medium Fees'
        else:
            return 'High Fees'
    if 'UP_FRONT_POINTS' in trade_info:
        points_value = float(trade_info['UP_FRONT_POINTS'])
        if points_value < 1:
            return 'Low Points'
        elif 1 <= points_value <= 2:
            return 'Medium Points'
        else:
            return 'High Points'
    return 'Uncategorized'

# Example usage
trade_description = "The APR is 5% and the up-front points are 2"
trade_info = extract_trade_info(trade_description)
category = categorize_trade(trade_info)
print(f"Trade Info: {trade_info}")
print(f"Category: {category}")


ValueError: [E955] Can't find table(s) lexeme_norm for language 'en' in spacy-lookups-data. Make sure you have the package installed or provide your own lookup tables if no default lookups are available for your language.